In [2]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/per_comune/"

In [7]:
sesso9701 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_1997_2001_per_sex.csv.csv')
eta0219 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2002_2019_per_age.csv.csv')
eta0211 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2002_2011_per_age.csv.csv')
sesso1924 = pd.read_csv(data_folder+'clean_data/popolazione_piemonte/population_2019_2024_per_sex.csv.csv')

In [20]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

---

In [26]:
confini.rename(columns={'Codice Com': 'Codice comune'})

,cartodb_id,comune,Codice comune,geometry
0,838,sanmartinoalfieri,A005099,"POLYGON ((8.10299 44.83612, 8.10399 44.83548, ..."
1,964,montecastello,A006105,"POLYGON ((8.68521 44.9705, 8.68913 44.97299, 8..."
2,667,priola,A004177,"POLYGON ((8.01499 44.279, 8.01754 44.27937, 8...."
3,341,civiasco,A002043,"POLYGON ((8.32285 45.83257, 8.32924 45.82994, ..."
4,7475,occhieppoinferiore,A096040,"POLYGON ((8.0254 45.55869, 8.02695 45.55694, 8..."
...,...,...,...,...
1201,151,meugliano,A001151,"MULTIPOLYGON (((7.70386 45.49981, 7.70237 45.4..."
1202,5,alicesuperiore,A001005,"MULTIPOLYGON (((7.80111 45.47364, 7.80056 45.4..."
1203,298,vicocanavese,A001297,"MULTIPOLYGON (((7.63404 45.57079, 7.64033 45.5..."
1204,138,lugnacco,A001138,"MULTIPOLYGON (((7.80456 45.4514, 7.80115 45.45..."


In [13]:
sesso9701.drop(columns=sesso9701.columns[[0,1]], inplace=True)

In [29]:
pop_97_01 = sesso9701.merge(confini.rename(columns={'Codice Com': 'Codice comune'}), on='Codice comune', how='inner')

In [31]:
pop_97_01.drop(columns=pop_97_01.columns[[0, 1, 10, 11]], inplace=True)

In [37]:
pop_97_01

,Sesso,Popolazione inizio periodo,Nati,Morti,Saldo migratorio interno,Saldo residuo,Popolazione fine periodo,Anno,geometry
0,Maschi,9281,62,149,22,-31,9185,1997,"POLYGON ((8.43848 44.71089, 8.43929 44.70894, ..."
1,Femmine,10375,67,149,58,-8,10343,1997,"POLYGON ((8.43848 44.71089, 8.43929 44.70894, ..."
2,Totale,19656,129,298,80,-39,19528,1997,"POLYGON ((8.43848 44.71089, 8.43929 44.70894, ..."
3,Maschi,198,1,7,4,-3,193,1997,"POLYGON ((9.13531 44.72286, 9.13755 44.71993, ..."
4,Femmine,198,3,5,2,-1,197,1997,"POLYGON ((9.13531 44.72286, 9.13755 44.71993, ..."
...,...,...,...,...,...,...,...,...,...
17455,Femmine,795,3,10,-6,6,788,2001,"POLYGON ((8.43327 45.41209, 8.43311 45.40937, ..."
17456,Totale,1621,8,15,3,7,1624,2001,"POLYGON ((8.43327 45.41209, 8.43311 45.40937, ..."
17457,Maschi,60,1,0,2,0,63,2001,"POLYGON ((8.1649 45.84672, 8.17064 45.84449, 8..."
17458,Femmine,74,0,0,0,2,76,2001,"POLYGON ((8.1649 45.84672, 8.17064 45.84449, 8..."


In [ ]:
dic_97_01 = dict.fromkeys(list(pop_97_01.Anno.unique()))
for a in dic_97_01.keys():
    dic_97_01[a] = pop_97_01[pop_97_01['Anno'] == a].drop(columns=['Anno'])
